## Making a small Chat application using sockets

With the advent of the internet, computer networks have undoubtedly become one of the most important areas of technology. At the heart of this digital communication technology lies the idea of sockets. At the application programming level, we can read and write from a specific socket object, identified by its IP address and port number, just like we would read and write from a file object.
The python `socket` library provides a clean interface for using these sockets for building our application.

In this notebook, we will explore the concept of sockets by building a small chat application using a server-client model.
At the end of it, the end-user will be able to sign into a chatroom and chat with different users signed into the room.

1. Design of the application
2. Using sockets
3. Using threads
4. Final Application

#### Prerequisites:

* Intermediate understanding of Python

### 1. Design of the Chat Application

For our final product, we want an application in which an end-user will be able to connect to a certain chatroom and have conversations with other end-users in the same chatroom. To achieve this, we will build a server application whose responsibility is to mediate the exchange of messages between all the participants in the chatroom, thus acting as "the chatroom", and another client application that the end-users will use to send and receive messages.

<img src="assets/chatroom.png">

### 2. Using sockets

Before we begin to write the application, let's get comfortable with using sockets first. Without using any abstractions like a class, we will explore it straight-up through scripting. Start a Python 3 interpreter. This script will act as the server; we'll refer to it as `server.py`.

In [1]:
import socket

#Create a socket
server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
print(server)    #Print the socket object

<socket.socket fd=39, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('0.0.0.0', 0)>


We have just created a socket object named server.

Next, we will bind it to an IP address and port, then start listening for incoming connections.

In [2]:
server.bind(('', 8100))    #Bind to any address available on port 8100
server.listen(2)           #Listen to maximum 2 connection

OSError: [Errno 98] Address already in use

Now the server is listening. That's all the code required to set up a server socket.

Next, we need to make a client socket.

Right now, the client socket can be written in the same script, but in order to really observe that the data is transported through network sockets and not through application memory, we will create another client-side script. Open another Python 3 interpreter for the client; we will refer to this script as `client.py`

In [ ]:
import socket

#Create a socket
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
print(client)    #Print the socket object

You can see that the process for creating a client-side socket is identical to creating a server-side socket. We simply create a socket object by calling `socket.socket()`.

Next, the client simply connects to the server.

In [ ]:
client.connect(('localhost', 8100))

The client is now connecting to the server.
Let's go back to `server.py` now.

In the server script, the server needs to "accept" an incoming connection.
we will do that by entering the following.

In [ ]:
(client_socket, client_address) = server.accept()
print(client_socket)
print(client_address)

the `accept()` method returns a tuple containing a socket object representing the client, and the address of the client. We will assign a name to the items in the tuple, namely `client_socket` and `client_address`, so that we can reuse these objects to do things later.

Let's make the server send a welcome message to the client to acknowledge that the connection was accepted.

In [ ]:
client_socket.sendall(b"Welcome to the chatroom")

To receive the welcome message, we need to return to `client.py`

In the client-side script, we receive the message by using the `recv()` method.

In [ ]:
greeting = client.recv(64)
print(greeting)

If you run both scripts at this point, the server will just send the greeting message and exit, and the client will simply receive the message and exit.

Code so far:

```
#server.py

import socket

#Create a socket
server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
print(server)    #Print the socket object

server.bind(('', 8100))    #Bind to any address available on port 8100
server.listen(2)           #Listen to maximum 2 connection

(client_socket, client_address) = server.accept()
print(client_socket)
print(client_address)

client_socket.sendall(b"Welcome to the chatroom")

```

```
#client.py

import socket

#Create a socket
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
print(client)    #Print the socket object

client.connect(('localhost', 8100))

greeting = client.recv(64)
print(greeting)
```

We now want to create some sort of a loop for sending and receiving data back and forth. In `client.py`, we add the following after receiving the greeting message:

```
while True:
    user_input = input('').encode('utf-8')
    client.sendall(user_input)
```

Now the client script will not terminate unless you interrupt the process by pressing `ctrl` + `c`. It will keep prompting the user for the next line of message to send, which is the behavior we want.

In `server.py`, we add the following after sending the greeting message:

```
while True:
    message = client_socket.recv(64)
    if not message:
        break
    print(message)
```

This will make the server script constantly receive messages from the client. After it receives a message, it will print the message and immediately wait to receive the next message.

At this point, the client will be able to send messages to the server. The server won't do anything interesting; it will simply print what it received.

Let's just echo the message back to the client. Modify `server.py` by adding a line `client_socket.sendall(message)` so that it looks like this:

```
while True:
    message = client_socket.recv(64)
    if not message:
        break
    print(message)
    client_socket.sendall(message)
```

The client won't print any replies yet, because we haven't added the code to receive the message.

Modify the while loop in `client.py` so that it looks like this:

```
while True:
    user_input = input('').encode('utf-8')
    client.sendall(user_input)
    reply = client.recv(64)
    print(reply)
```

Now the server and client will maintain connection, and the client will be able to send messages to the server and get them echoed back.

With both scripts running, if we run another instance of client.py, this second client won't be able to send and receive the messages back.
This is because the server process is "trapped" inside the while loop running for the first client.

<img src="assets/chat_screen1.png">

We should modify the server script so that it can run multiple while loops at the same time. This is where we need to use threading.

*The code so far can be found [here (server)](scripts/simple_server.py) and [here (client)](scripts/simple_client.py)

### 3. Using threads

Now that we've covered the basics of using the python socket library, we will modify the scripts we created so that it looks more like a chat application. So far, the server can only handle one client at a time while our application requires that it maintains multiple connections simultaneously. There is a while loop in our client-handling code that does not break unless the client disconnects and this is causing the application to block, preventing the server from accepting the next incoming connnection. However, we need this while loop for every client connecting to the server. This is where threading can be useful, or even necessary.

We will start a child thread for each client in the chatroom. We won't need to worry about thread joins and mutexes here because our child processes can run completely concurrently. In a liberal chatroom like ours, we don't have a [talking stick](https://en.wikipedia.org/wiki/Talking_stick) policy; anyone can say things at any time.

By keeping a list of clients, the server will be able to pass information between the threads.
Right after the server starts listening, create an empty list named `clients`.

```
#Create an empty list of clients
clients = []

```

Next, we will create a subclass of `threading.Thread` to describe what will run on each thread. `threading.Thread` has a simple API for basic usage:

1. Define a new class by subclassing threading.Thread.
2. Override `__init__` method of the Thread class, passing any thread-specific data as arguments.
3. Override `run` method of the Thread class. This is the body of the thread.
4. Create an instance of the newly created class.
5. Call `start()` to start the thread.

```
class ClientHandler(threading.Thread):
    def __init__(self, client_socket):
        threading.Thread.__init__(self)
        self.socket = client_socket
        clients.append(self)

    def run(self):
        self.socket.sendall(b"Welcome to the chatroom")
        while True:
            message = self.socket.recv(64)
            if not message:
                break
            print(message)
            self.socket.sendall(message)
        clients.remove(self)
        self.socket.close()

```

We define a class like shown above. The client's socket object will be passed as the argument to the constructor and the thread object will store it as an attribute. Upon creation, this thread will be added to the `clients` list. Inside the `run()` method, the socket object is then accessed. The `run()` method contains the code to send the greetings message, then constantly receive messages one after another, echoing back whenever it receives one.

We can now make the server keep listening for incoming connections, accepting and starting a child thread each time a new client connects.

```
while True:
    (client_socket, client_address) = server.accept()
    handler = ClientHandler(client_socket)
    handler.start()
server.close()
```

We create a new thread object (ClientHandler), and call `start()` to start the thread. 

If you run the server script now, and run two instances of the client script, both will be able to connect, send messages and get them echoed back. Now we will modify the run method of the ClientHandler so that it broadcasts whatever it received to the rest of the clients.
Remove or comment out the line responsible for echoing messages back: `self.socket.sendall(message)`

```
def run(self):
        self.socket.sendall(b"Welcome to the chatroom")
        while True:
            message = self.socket.recv(64)
            if not message:
                break
            print(message)
            for client in clients:
                if not client == self:
                    client.socket.sendall(message)
        clients.remove(self)
        self.socket.close()
```

Upon receiving a message, the program will go through the list of clients, and send the message to the client if it isn't the client that sent the message. The server script so far can be found [here](scripts/threaded_server.py)

If you run the server script now with multiple clients and play around for a little bit, you will notice a strange behavior on the client terminal. It looks as though the broadcast messages pile up in a queue, and the client can read the next message only after sending one message. And the server won't print the messages unless all the clients have sent the same number of messages.

<img src="assets/chat_screen2.png">

This weird behavior happens because our client script waits to receive a message before sending one. As shown in the code below, the client waits for the `recv()` after `sendall()`.

```
while True:
    user_input = input('').encode('utf-8')
    client.sendall(user_input)
    reply = client.recv(64)
    print(reply)
```

We will now fix the problem by using another thread in the client script dedicated to handling the receives. The main thread will be used for sending messages, while the second thread will be used for receiving messages.

```
class ReceiveHandler(threading.Thread):
    def __init__(self, socket):
        threading.Thread.__init__(self)
        self.socket = socket
        
    def run(self):
        while True:
            message = self.socket.recv(64)
            if not message:
                continue
            print(message.decode('utf-8'))
```

We define a subclass of `threading.Thread` like shown above. The `run()` method contains a while loop and all there is in it is the code to receive a message and print it. We start this thread before we start our main loop.

```
receive_handler = ReceiveHandler(client)
receive_handler.start()
```

Now that the client application can receive messages on a separate thread, we can start the main loop for sending messages. We remove the subsequent `recv()` after the `sendall()` call, which was the line that was causing the strange blocking behavior.

```
while True:
    user_input = input('').encode('utf-8')
    client.sendall(user_input)
client.close()

```

The code for this threaded version of the client can be found [here](scripts/threaded_client.py)

### 4. Final Application

What we have so far is complete in terms of basic functionality. The server is able to receive and broadcast messages to all the participants, and the clients are able to send messages and receive messages other people are sending.

However, we could definitely improve the interface a little; right now we have no way of telling who is sending which message. Only the messages are exchanged, but no information about the participants are shared.

We will modify the code so that the server will generate some random name for each client, and prepend the client's name before every message broadcast. We import python modules `random` and `string` to define a small function that generates random string of length 8.

```
import random, string

def randname():
    return ''.join(random.SystemRandom().choice(string.ascii_letters) for _ in range(8))
    
```

Then in the `__init__` method of the client's Thread, we will assign a name by calling the `randname()` function we just defined.

```
class ClientHandler(threading.Thread):
    def __init__(self, client_socket, client_address):
        threading.Thread.__init__(self)
        self.socket = client_socket
        self.name = randname()
```

We then prepend `self.name` to the message that will be broadcast to all the clients.

```
    def run(self):
        self.socket.sendall(b"Welcome to the chatroom "+self.name.encode("utf-8"))
        while True:
            message = self.socket.recv(64)
            if not message:
                break
            print(self.name+" : "+message.decode("utf-8"))
            for client in clients:
                if not client == self:
                    client.socket.sendall(self.name.encode("utf-8")+b" : "+message)
        clients.remove(self)
        self.socket.close()

```

There are many more improvements that can be made to the application, such as a "sign in" routine, but that is outside the scope of this tutorial. We shall leave it up to the readers to take the application further and make it more interesting.

The final version of the scripts can be found [here (server)](scripts/chat_server.py) and [here (client)](scripts/chat_client.py)

* [simple_server.py](scripts/simple_server.py)
* [simple_client.py](scripts/simple_client.py)
* [threaded_server.py](scripts/threaded_server.py)
* [threaded_client.py](scripts/threaded_client.py)
* [chat_server.py](scripts/chat_server.py)
* [chat_client.py](scripts/chat_client.py)